# Fix pathing

In [1]:
import sys


sys.path.append("..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


# Init

In [3]:
import datahandler

import matplotlib.pyplot as plt


dataset_id = "oslo"

data_preprocessor = datahandler.DataPreprocessorOUS(dataset_id)
data_preprocessor.execute()

data_loader = datahandler.DataLoader(dataset_id)
data_loader.execute()


Loading dataset: 100%|██████████| 6/6 [00:22<00:00,  3.77s/it]


# Analysis
- All of the rows where "time_available" is NaN should be dropped (64 rows)
- All of the rows where "time_dispatch" is NaN should be dropped (111 rows)
- All of the rows where "triage_impression_during_call" is NaN should be dropped (9 rows)
- All of the rows where "time_arrival_scene" is NaN and "time_arrival_hospital" is not NaN should be dropped (13695 rows)
- All of the rows where "time_departure_scene" is NaN and "time_arrival_hospital" is not NaN should be dropped (8356 rows)


In [4]:
nan_rows = data_loader.processed_incidents_df[data_loader.processed_incidents_df["time_available"].isna()]
print(nan_rows.shape[0])


64


In [5]:
nan_rows = data_loader.processed_incidents_df[data_loader.processed_incidents_df["time_dispatch"].isna()]
print(nan_rows.shape[0])

23104


In [6]:
nan_rows = data_loader.processed_incidents_df[data_loader.processed_incidents_df["triage_impression_during_call"].isna()]
print(nan_rows.shape[0])

9


In [7]:
count = data_loader.processed_incidents_df[(data_loader.processed_incidents_df["time_arrival_scene"].isna()) & (~data_loader.processed_incidents_df["time_arrival_hospital"].isna())].shape[0]
print(count)

13695


In [8]:
count = data_loader.processed_incidents_df[(data_loader.processed_incidents_df["time_departure_scene"].isna()) & (~data_loader.processed_incidents_df["time_arrival_hospital"].isna())].shape[0]
print(count)

8356


In [9]:
print(data_loader.enhanced_incidents_df.shape)

(711924, 19)


In [10]:
print(data_loader.enhanced_incidents_df["response_time_sec"])

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
           ...  
711919    1125.0
711920    1773.0
711921    1449.0
711922    5527.0
711923    2357.0
Name: response_time_sec, Length: 711924, dtype: float64


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

df = data_loader.enhanced_incidents_df

# Determine the extents of your map from your dataframe
min_longitude = df['longitude'].min()
max_longitude = df['longitude'].max()
min_latitude = df['latitude'].min()
max_latitude = df['latitude'].max()

grid_width = 1000  # assuming each grid cell is 1000m width, adjust as needed
grid_height = 1000  # assuming each grid cell is 1000m height, adjust as needed

# Create a map with grid lines
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent([min_longitude, max_longitude, min_latitude, max_latitude])

# Add natural features
ax.add_feature(cfeature.LAND, edgecolor='black')
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Overlay grid based on easting and northing values
easting_vals = np.arange(df['easting'].min(), df['easting'].max(), grid_width)
northing_vals = np.arange(df['northing'].min(), df['northing'].max(), grid_height)

for east in easting_vals:
    ax.plot([east, east], [df['northing'].min(), df['northing'].max()], color="red", transform=ccrs.PlateCarree())
for north in northing_vals:
    ax.plot([df['easting'].min(), df['easting'].max()], [north, north], color="red", transform=ccrs.PlateCarree())

# Plotting the incident locations from the dataframe
ax.scatter(df['longitude'], df['latitude'], color='blue', transform=ccrs.PlateCarree())

plt.show()